# Oversampling 6 Time Bellweathers SVM

## Import Libaries

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

### Import & Clean Dataset

In [36]:
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

In [37]:
df = pd.read_csv('data_dropna.csv')

In [38]:
df.head()

Unnamed: 0  fips    state    county  wins  poverty_rate_1960  \
0           6  1011  Alabama   Bullock     2               72.2   
1           7  1013  Alabama    Butler     3               63.3   
2           8  1015  Alabama   Calhoun     3               34.1   
3           9  1017  Alabama  Chambers     3               42.9   
4          10  1019  Alabama  Cherokee     3               47.2   

   poverty_rate_1970  poverty_rate_1980  poverty_rate_1990  poverty_rate_2000  \
0               54.9               35.4               36.5               33.5   
1               40.8               27.2               31.5               24.6   
2               20.9               16.3               15.7               16.1   
3               25.5               17.8               18.8               17.0   
4               24.0               18.6               17.6               15.6   

   poverty_rate_2010  population_1960  population_1970  population_1980  \
0               22.2            13402            11817            10565   
1               25.7            24545            21903            21423   
2               21.4            92644            97822           111618   
3               23.3            38750            36096            38869   
4               19.5            16875            15536            18668   

   population_1990  population_2000  population_2010  persons_in_poverty_1960  \
0            10340            10169            10313                     9680   
1            21648            21143            20463                    15530   
2           110981           109830           114887                    31601   
3            36424            36006            33785                    16604   
4            19464            23601            25440                     7966   

   persons_in_poverty_1970  persons_in_poverty_1980  persons_in_poverty_1990  \
0                     6492                     3745                     3776   
1                     8934                     5818                     6815   
2                    20395                    18231                    17385   
3                     9214                     6931                     6849   
4                     3731                     3474                     3424   

   persons_in_poverty_2000  persons_in_poverty_2010  \
0                     3405                     2286   
1                     5205                     5256   
2                    17695                    24597   
3                     6134                     7863   
4                     3672                     4956   

   rural_urban_continuum_code_2013  urban_influence_code_2013  metro_2013  \
0                                2                          2           1   
1                                3                          2           1   
2                                6                          6           0   
3                                1                          1           1   
4                                1                          1           1   

   civilian_labor_force_2000  civilian_labor_force_2001  \
0                      21720                      21955   
1                      69533                      69161   
2                      11373                      11250   
3                       8565                       9081   
4                      25106                      25305   

   civilian_labor_force_2002  civilian_labor_force_2003  \
0                      22094                      22604   
1                      69169                      72299   
2                      10971                      10977   
3                       8933                       8836   
4                      25757                      25900   

   civilian_labor_force_2004  civilian_labor_force_2005  \
0                      23218                      23949   
1                      74772                      76804   
2            

In [39]:
df = df.drop(['Unnamed: 0'], axis=1)
len(df['wins'])

2974

In [40]:
# Add 2020 bellwether code (here we specify wins to 6)
# Specifying a bellweather to be either 5 or 6 wins would result in 59 bellweathers under the new classification

binary_bellweathers = []
for i, v in enumerate(df['wins']):
    if df.iloc[i, 3] == 6:
        binary_bellweathers.append(1)
    elif df.iloc[i, 3] < 6:
        binary_bellweathers.append(0)

In [41]:
binary_bellweathers.count(1)

9

In [42]:
# gets rid of irrelevant columns, strings, and wins (since that would give the model 100% accuracy)
df = df.drop(['fips', 'state', 'county', 'wins'], axis=1)
df.head()

poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  poverty_rate_1990  \
0               72.2               54.9               35.4               36.5   
1               63.3               40.8               27.2               31.5   
2               34.1               20.9               16.3               15.7   
3               42.9               25.5               17.8               18.8   
4               47.2               24.0               18.6               17.6   

   poverty_rate_2000  poverty_rate_2010  population_1960  population_1970  \
0               33.5               22.2            13402            11817   
1               24.6               25.7            24545            21903   
2               16.1               21.4            92644            97822   
3               17.0               23.3            38750            36096   
4               15.6               19.5            16875            15536   

   population_1980  population_1990  population_2000  population_2010  \
0            10565            10340            10169            10313   
1            21423            21648            21143            20463   
2           111618           110981           109830           114887   
3            38869            36424            36006            33785   
4            18668            19464            23601            25440   

   persons_in_poverty_1960  persons_in_poverty_1970  persons_in_poverty_1980  \
0                     9680                     6492                     3745   
1                    15530                     8934                     5818   
2                    31601                    20395                    18231   
3                    16604                     9214                     6931   
4                     7966                     3731                     3474   

   persons_in_poverty_1990  persons_in_poverty_2000  persons_in_poverty_2010  \
0                     3776                     3405                     2286   
1                     6815                     5205                     5256   
2                    17385                    17695                    24597   
3                     6849                     6134                     7863   
4                     3424                     3672                     4956   

   rural_urban_continuum_code_2013  urban_influence_code_2013  metro_2013  \
0                                2                          2           1   
1                                3                          2           1   
2                                6                          6           0   
3                                1                          1           1   
4                                1                          1           1   

   civilian_labor_force_2000  civilian_labor_force_2001  \
0                      21720                      21955   
1                      69533                      69161   
2                      11373                      11250   
3                       8565                       9081   
4                      25106                      25305   

   civilian_labor_force_2002  civilian_labor_force_2003  \
0                      22094                      22604   
1                      69169                      72299   
2                      10971                      10977   
3                       8933                       8836   
4                      25757                      25900   

   civilian_labor_force_2004  civilian_labor_force_2005  \
0                      23218                      23949   
1                      74772                      76804   
2                      10633                      10760   
3                       8843                       8861   
4                      26208                      26446   

   civilian_labor_force_2006  civilian_labor_force_2007  \
0                      24398                      24383   
1     

In [43]:
df.insert(0, "bellweather?", binary_bellweathers)

In [44]:
df.head()

bellweather?  poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  \
0             0               72.2               54.9               35.4   
1             0               63.3               40.8               27.2   
2             0               34.1               20.9               16.3   
3             0               42.9               25.5               17.8   
4             0               47.2               24.0               18.6   

   poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  population_1960  \
0               36.5               33.5               22.2            13402   
1               31.5               24.6               25.7            24545   
2               15.7               16.1               21.4            92644   
3               18.8               17.0               23.3            38750   
4               17.6               15.6               19.5            16875   

   population_1970  population_1980  population_1990  population_2000  \
0            11817            10565            10340            10169   
1            21903            21423            21648            21143   
2            97822           111618           110981           109830   
3            36096            38869            36424            36006   
4            15536            18668            19464            23601   

   population_2010  persons_in_poverty_1960  persons_in_poverty_1970  \
0            10313                     9680                     6492   
1            20463                    15530                     8934   
2           114887                    31601                    20395   
3            33785                    16604                     9214   
4            25440                     7966                     3731   

   persons_in_poverty_1980  persons_in_poverty_1990  persons_in_poverty_2000  \
0                     3745                     3776                     3405   
1                     5818                     6815                     5205   
2                    18231                    17385                    17695   
3                     6931                     6849                     6134   
4                     3474                     3424                     3672   

   persons_in_poverty_2010  rural_urban_continuum_code_2013  \
0                     2286                                2   
1                     5256                                3   
2                    24597                                6   
3                     7863                                1   
4                     4956                                1   

   urban_influence_code_2013  metro_2013  civilian_labor_force_2000  \
0                          2           1                      21720   
1                          2           1                      69533   
2                          6           0                      11373   
3                          1           1                       8565   
4                          1           1                      25106   

   civilian_labor_force_2001  civilian_labor_force_2002  \
0                      21955                      22094   
1                      69161                      69169   
2                      11250                      10971   
3                       9081                       8933   
4                      25305                      25757   

   civilian_labor_force_2003  civilian_labor_force_2004  \
0                      22604                      23218   
1                      72299                      74772   
2                      10977                      10633   
3                       8836                       8843   
4                      25900                      26208   

   civilian_labor_force_2005  civilian_labor_force_2006  \
0                      23949                      24398   
1                      76804                      79711   
2                      10

In [45]:
df["bellweather?"].value_counts()

0    2965
1       9
Name: bellweather?, dtype: int64

In [46]:
# only take the 9 bellweathers

df1 = df[df["bellweather?"] == 1]

In [47]:
df1

bellweather?  poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  \
268              1               24.3               15.4               13.6   
1232             1               15.6               11.6               11.0   
1501             1               34.7               31.4               24.1   
1726             1               25.8               14.0               13.4   
1755             1               17.2                8.3                8.0   
2685             1               37.0               18.0                9.3   
2710             1                7.0                4.2                3.9   
2798             1               16.8               11.3               10.2   
2896             1               21.9               12.0                6.9   

      poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  \
268                11.3               10.7               12.4   
1232               15.9               13.2               12.5   
1501               27.7               28.1               27.1   
1726               12.3               11.6               12.4   
1755                5.9                5.7                6.5   
2685                7.0                7.1                8.8   
2710                3.5                4.5                5.6   
2798               12.5               12.5               13.5   
2896                9.8                6.4               11.4   

      population_1960  population_1970  population_1980  population_1990  \
268             60412            76823            93956           106673   
1232            37561            42982            44851            46282   
1501             7887             6600             6878             6596   
1726            34362            34353            35454            34767   
1755            87685           119161           150699           176816   
2685            30853            26410            34623            45305   
2710           252665           444679           589302           803636   
2798            29001            34192            50743            54705   
2896            20944            19876            24612            25318   

      population_2000  population_2010  persons_in_poverty_1960  \
268            122717           157929                    14668   
1232            47338            54900                     5858   
1501             6945             6425                     2733   
1726            35998            36728                     8865   
1755           196387           215731                    15093   
2685            59943            68377                    11401   
2710           958484          1074652                    17680   
2798            62602            69886                     4875   
2896            27555            27338                     4585   

      persons_in_poverty_1970  persons_in_poverty_1980  \
268                     11813                    12797   
1232                     4966                     4941   
1501                     2073                     1659   
1726                     4802                     4760   
1755                     9916                    12058   
2685                     4745                     3207   
2710                    18619                    23092   
2798                     3856                     5172   
2896                     2391                     1703   

      persons_in_poverty_1990  persons_in_poverty_2000  \
268                     12071                    13083   
1232                     7355                     6272   
1501                     1825                     1951   
1726                     4263                     4173   
1755                    10509                    11238   
2685                     3162                     4263   
2710                    28210                    43396   
2798                     6852                     7825   
2896                     2487

In [48]:
# increasing the number of bellweathers up to 1287, to give the dataset a roughly 30 - 70 split

df1_repeated = pd.concat([df1]*143, ignore_index=True)

In [49]:
df1_repeated.head()

bellweather?  poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  \
0             1               24.3               15.4               13.6   
1             1               15.6               11.6               11.0   
2             1               34.7               31.4               24.1   
3             1               25.8               14.0               13.4   
4             1               17.2                8.3                8.0   

   poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  population_1960  \
0               11.3               10.7               12.4            60412   
1               15.9               13.2               12.5            37561   
2               27.7               28.1               27.1             7887   
3               12.3               11.6               12.4            34362   
4                5.9                5.7                6.5            87685   

   population_1970  population_1980  population_1990  population_2000  \
0            76823            93956           106673           122717   
1            42982            44851            46282            47338   
2             6600             6878             6596             6945   
3            34353            35454            34767            35998   
4           119161           150699           176816           196387   

   population_2010  persons_in_poverty_1960  persons_in_poverty_1970  \
0           157929                    14668                    11813   
1            54900                     5858                     4966   
2             6425                     2733                     2073   
3            36728                     8865                     4802   
4           215731                    15093                     9916   

   persons_in_poverty_1980  persons_in_poverty_1990  persons_in_poverty_2000  \
0                    12797                    12071                    13083   
1                     4941                     7355                     6272   
2                     1659                     1825                     1951   
3                     4760                     4263                     4173   
4                    12058                    10509                    11238   

   persons_in_poverty_2010  rural_urban_continuum_code_2013  \
0                    19550                                1   
1                     6851                                9   
2                     1740                                9   
3                     4548                                3   
4                    13922                                1   

   urban_influence_code_2013  metro_2013  civilian_labor_force_2000  \
0                          1           1                     435261   
1                         12           0                       2782   
2                         10           0                       5498   
3                          2           1                      42544   
4                          1           1                      20207   

   civilian_labor_force_2001  civilian_labor_force_2002  \
0                     433800                     434196   
1                       2848                       2879   
2                       5625                       5736   
3                      42367                      42269   
4                      19926                      19938   

   civilian_labor_force_2003  civilian_labor_force_2004  \
0                     435069                     431901   
1                       2833                       2807   
2                       5801                       5758   
3                      41187                      40515   
4                      19920                      19858   

   civilian_labor_force_2005  civilian_labor_force_2006  \
0                     437887                     445627   
1                       2759                       2808   
2                       5

In [50]:
# take all of the non-bellweathers and store them in df2

df2 = df[df["bellweather?"] == 0]

In [51]:
df2["bellweather?"].value_counts()

0    2965
Name: bellweather?, dtype: int64

In [52]:
df_new = df1_repeated.append(df2)

In [54]:
df_new["bellweather?"].value_counts()

0    2965
1    1287
Name: bellweather?, dtype: int64

In [56]:
len(df_new)

4252

In [57]:
# store all features except wins in x_new
# store wins in y_new

x_new = df_new.loc[:, df_new.columns != 'bellweather?']
y_new = df_new["bellweather?"]

In [58]:
x_new.head()

poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  poverty_rate_1990  \
0               24.3               15.4               13.6               11.3   
1               15.6               11.6               11.0               15.9   
2               34.7               31.4               24.1               27.7   
3               25.8               14.0               13.4               12.3   
4               17.2                8.3                8.0                5.9   

   poverty_rate_2000  poverty_rate_2010  population_1960  population_1970  \
0               10.7               12.4            60412            76823   
1               13.2               12.5            37561            42982   
2               28.1               27.1             7887             6600   
3               11.6               12.4            34362            34353   
4                5.7                6.5            87685           119161   

   population_1980  population_1990  population_2000  population_2010  \
0            93956           106673           122717           157929   
1            44851            46282            47338            54900   
2             6878             6596             6945             6425   
3            35454            34767            35998            36728   
4           150699           176816           196387           215731   

   persons_in_poverty_1960  persons_in_poverty_1970  persons_in_poverty_1980  \
0                    14668                    11813                    12797   
1                     5858                     4966                     4941   
2                     2733                     2073                     1659   
3                     8865                     4802                     4760   
4                    15093                     9916                    12058   

   persons_in_poverty_1990  persons_in_poverty_2000  persons_in_poverty_2010  \
0                    12071                    13083                    19550   
1                     7355                     6272                     6851   
2                     1825                     1951                     1740   
3                     4263                     4173                     4548   
4                    10509                    11238                    13922   

   rural_urban_continuum_code_2013  urban_influence_code_2013  metro_2013  \
0                                1                          1           1   
1                                9                         12           0   
2                                9                         10           0   
3                                3                          2           1   
4                                1                          1           1   

   civilian_labor_force_2000  civilian_labor_force_2001  \
0                     435261                     433800   
1                       2782                       2848   
2                       5498                       5625   
3                      42544                      42367   
4                      20207                      19926   

   civilian_labor_force_2002  civilian_labor_force_2003  \
0                     434196                     435069   
1                       2879                       2833   
2                       5736                       5801   
3                      42269                      41187   
4                      19938                      19920   

   civilian_labor_force_2004  civilian_labor_force_2005  \
0                     431901                     437887   
1                       2807                       2759   
2                       5758                       5801   
3                      40515                      40595   
4                      19858                      19959   

   civilian_labor_force_2006  civilian_labor_force_2007  \
0                     445627                     452749   
1     

In [59]:
y_new.value_counts()

0    2965
1    1287
Name: bellweather?, dtype: int64

In [61]:
x_train_new, x_test_new, y_train_new, y_test_new = \
    train_test_split(x_new, y_new, test_size=0.25)

np.unique(y_new)

array([0, 1], dtype=int64)

In [62]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(x_train_new, y_train_new)

SVC(kernel='linear')

In [63]:
y_pred_new = svclassifier.predict(x_test_new)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test_new,y_pred_new))
print(classification_report(y_test_new,y_pred_new))

[[719   8]
 [ 35 301]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       727
           1       0.97      0.90      0.93       336

    accuracy                           0.96      1063
   macro avg       0.96      0.94      0.95      1063
weighted avg       0.96      0.96      0.96      1063

